In [2]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run, ir_datasets
from tira.rest_api_client import Client
import pyterrier as pt
from pyterrier_pisa import PisaIndex

In [3]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)
print("Files in IR-ACL corpus: %s " % len(list(pt_dataset.get_corpus_iter())))


# We are using Pisa Index to index the dataset
#index = PisaIndex('./index', overwrite=True)
#index.index(pt_dataset.get_corpus_iter())

# get all topics of training dataset
#topics = pt_dataset.get_topics()
#topics

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:   0%|          | 0/126958 [00:00<?, ?it/s]

Files in IR-ACL corpus: 126958 


# We now do Query expansion in order to improve retrieval effectiveness 
Query expansion generally improves recall, by adding more terms to the query, it broadens the search scope, potentially retrieving more relevant documents. While it could also have a slight negative effect on precision since query expansion might also introdoces irrelevant results. 

In [5]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [11]:
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

bo1_pipe = bm25 >> pt.rewrite.Bo1QueryExpansion(index) >> bm25
pipelineDisplay = bm25 >> bo1_pipe
pipelineDisplay.search("retrieval")
pt.Experiment([bm25, bo1_pipe], pt_dataset.get_topics(), pt_dataset.get_qrels(), eval_metrics=['ndcg_cut_5'], names=['BM25', 'BM25 >> Bo1 >> BM25'])

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,ndcg_cut_5
0,BM25,0.393650
1,BM25 >> Bo1 >> BM25,0.381675


In [6]:
kl_pipe = bm25 >> pt.rewrite.KLQueryExpansion(index) >> bm25

pt.Experiment([bm25, kl_pipe], pt_dataset.get_topics(), pt_dataset.get_qrels(), eval_metrics=['ndcg_cut_5'], names=['BM25', 'BM25 >> KL >> BM25'])

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,ndcg_cut_5
0,BM25,0.393650
1,BM25 >> KL >> BM25,0.383947


In [8]:
if not pt.started():
    pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

rm3_pipe = bm25 >> pt.rewrite.RM3(index) >> bm25

pt.Experiment([bm25, rm3_pipe], pt_dataset.get_topics(), pt_dataset.get_qrels(), eval_metrics=['ndcg_cut_5'], names=['BM25', 'BM25 >> RM3 >> BM25'])

There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.
10:40:58.684 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 125137 among 5 possibilities
10:40:58.869 [main] WARN org.terrier.querying.RM1 - Did not identify any usable candidate expansion terms from docid 116910 among 4 possibilities


,name,ndcg_cut_5
0,BM25,0.393650
1,BM25 >> RM3 >> BM25,0.341725
